In [18]:
import numpy as np
from sklearn.decomposition import PCA
import os
import pickle

In [19]:
folder = '/home/lyt/code/vqa-concept/dataTVQA/image-feature/bottomup'

In [20]:
trainfile = '{}/ocr_bert.new.{}/{}_50_ocr.0init.npy'.format(folder, 768, 'train')
valfile = '{}/ocr_bert.new.{}/{}_50_ocr.0init.npy'.format(folder, 768, 'val')
testfile = '{}/ocr_bert.new.{}/{}_50_ocr.0init.npy'.format(folder, 768, 'test')
itoafile = '{}/ocr_bert.new.{}/{}_ocr.npy'.format(folder, 768, 'itoa')

In [21]:
trainX = np.load(trainfile)
valX = np.load(valfile)
testX = np.load(testfile)
itoaX = np.load(itoafile)

In [22]:
print(trainX.shape)
print(valX.shape)
print(testX.shape)
print(itoaX.shape)

(21953, 50, 768)
(3166, 50, 768)
(3289, 50, 768)
(9452, 768)


In [23]:
trainX = trainX.reshape(-1,768)
valX = valX.reshape(-1,768)
testX = testX.reshape(-1,768)

In [24]:
print(trainX.shape)
print(valX.shape)
print(testX.shape)
print(itoaX.shape)

(1097650, 768)
(158300, 768)
(164450, 768)
(9452, 768)


In [25]:
trainZeroIdx = np.sum(np.abs(trainX), 1) == 0
valZeroIdx = np.sum(np.abs(valX), 1) == 0
testZeroIdx = np.sum(np.abs(testX), 1) == 0

In [26]:
#X = np.concatenate((trainX, valX, itoaX), 0) # train on Tr; test on Val
X = np.concatenate((trainX, valX, testX, itoaX), 0) # train on Tr+Val; test on Te
print(X.shape)
nonzeroIdx = np.sum(np.abs(X), 1) != 0
X = X[nonzeroIdx]
print(X.shape)

(1429852, 768)
(286879, 768)


In [27]:
pcafile = '{}/ocr_bert.new.{}/pca.0init.remain0.pkl'.format(folder, 300)
if os.path.exists(pcafile):
    with open(pcafile, 'rb') as f:
        pca=pickle.load(f)
else:
    pca = PCA(n_components=300)
    pca.fit(X)
    with open(pcafile, 'wb') as f:
        pickle.dump(pca, f)

In [28]:
trX = pca.transform(trainX)
vaX = pca.transform(valX)
teX = pca.transform(testX)
iaX = pca.transform(itoaX)

In [29]:
print(trX.shape)
print(vaX.shape)
print(teX.shape)
print(iaX.shape)

(1097650, 300)
(158300, 300)
(164450, 300)
(9452, 300)


In [30]:
# remain 0 if original is 0
trX[trainZeroIdx] = 0
vaX[valZeroIdx] = 0
teX[testZeroIdx] = 0

In [31]:
trX = trX.reshape(-1,50,300)
vaX = vaX.reshape(-1,50,300)
teX = teX.reshape(-1,50,300)

In [32]:
print(trX.shape)
print(vaX.shape)
print(teX.shape)
print(iaX.shape)

(21953, 50, 300)
(3166, 50, 300)
(3289, 50, 300)
(9452, 300)


In [33]:
trainfile = '{}/ocr_bert.new.{}/{}_50_ocr.0init.remain0.npy'.format(folder, 300, 'train')
valfile = '{}/ocr_bert.new.{}/{}_50_ocr.0init.remain0.npy'.format(folder, 300, 'val')
testfile = '{}/ocr_bert.new.{}/{}_50_ocr.0init.remain0.npy'.format(folder, 300, 'test')
itoafile = '{}/ocr_bert.new.{}/{}_ocr.0init.remain0.npy'.format(folder, 300, 'itoa')

In [34]:
np.save(trainfile,trX)
np.save(valfile,vaX)
np.save(testfile,teX)
np.save(itoafile,iaX)